In [82]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt
from tqdm import tqdm
import timeit
# set the working directory
BASE_DIR = Path.cwd()
# define the exported folder path
# Check if folder exists
folder_path = pathlib.Path(BASE_DIR.parent.joinpath("Exported_Files","census_tract","agg_network","Feb242022","CensusTract_FT_FullNetwork"))
folder_path.mkdir(parents=True, exist_ok=True)
# print(BASE_DIR)

In [83]:
# """
# Fetch SF Champ network both for year 2010, year 2016 and join the frames
# """
# # Keep only FT types representing real road-network
# def get_realnetwork(_df):
#     """
#     Keep only FT types representing real road-network
#     1: Fwy-Fwy Connector; 2: Freeway; 3: Expressway; 4: Collector; 5: Ramp; 6: Centroid Connector;
#     7: Major Arterial; 8: ; 9: Alley (only for DTA); 10: ; 11: Local; 12: Minor Arterial; 13: Bike only;
#     14: ; 15: Super Arterial
#     :param _df:
#     :return df:
#     """
#     df = _df.copy()
#     if isinstance(df,gpd.GeoDataFrame):
#         if "FT" in df.columns:
#             df = df.loc[df["FT"].isin([1,2,3,4,5,7,9,10,11,12,13,15])]
#     return df
# # create a new field to store Time-of-Day information
# def add_TOD_information(_df,TOD):
#     """
#     before concatenating dataframes, insert the TOD information as column value
#     :param _df:
#     :param TOD:
#     :return:
#     """
#     d = {}
#     if isinstance(_df,gpd.GeoDataFrame):
#         d["peak"] = TOD
#     return pd.concat([_df, pd.DataFrame(d, index=_df.index)],axis=1)
#
# def clip_roadnetwork(_dfroadnetwork):
#     """
#     Overlay and clip the line features to get only features inside SF County Area
#     :param _dfroadnetwork:
#     :return:
#     """
#     dfPolygon = gpd.read_file(BASE_DIR.parent.joinpath("Data","SF_County","SFBay_Boundary.shp"))
#     dfPolygon=dfPolygon.to_crs("EPSG:4326")
#     dfroadnetwork = _dfroadnetwork.to_crs(4326).copy()
#     df = gpd.clip(dfroadnetwork,dfPolygon)
#     return df
#
# def getTotalCapacity(_df):
#     """
#     The "CAP" field is for per/hour; convert this to TOD period
#     :param _df:
#     :return:
#     """
#     df = _df.copy()
#     df["Tot_CAP"] = df["CAP"]
#     df.loc[(df["peak"]=="EV"),"Tot_CAP"]*=8.5
#     df.loc[(df["peak"]=="MD"),"Tot_CAP"]*=6.5
#     df.loc[(df["peak"]=="AM") | (df["peak"]=="PM") |(df["peak"]=="EA") ,"Tot_CAP"]*=3
#     return df
#
# def check_required_rdntwrk_colmns(_df):# road network
#     df = _df.copy()
#     d = {}
#     reqd_colmns = ['V1_1', 'V2_1', 'V3_1', 'V4_1', 'V5_1', 'V6_1', 'V7_1', 'V8_1', 'V9_1', 'V10_1',
#                    'V11_1', 'V12_1',"V13_1",'V14_1', 'V15_1',"V16_1",'V17_1', 'V18_1',"V19_1",
#                    'OOS', 'PUDO','Tot_Vol',"Tot_TNC_Vol", "Tot_Non_TNC_Vol",
#                    'BUSVOL_AM', 'BUSVOL_AM', 'BUSVOL_MD', 'BUSVOL_PM', 'BUSVOL_EV', 'BUSVOL_EA',
#                    ]
#     for col in reqd_colmns:
#         if col not in df.columns:
#             d[col]=0
#     return pd.concat([df, pd.DataFrame(d, index=df.index)],axis=1)
#
# def merge_TOD_dfs(_dfAM,_dfPM,_dfEA,_dfEV,_dfMD):
#     # clean the dataframe
#     dfAM = get_realnetwork(_dfAM.copy())
#     dfPM = get_realnetwork(_dfPM.copy())
#     dfEA = get_realnetwork(_dfEA.copy())
#     dfEV = get_realnetwork(_dfEV.copy())
#     dfMD = get_realnetwork(_dfMD.copy())
#     # clip road network
#     dfAM = clip_roadnetwork(dfAM.copy())
#     dfPM = clip_roadnetwork(dfPM.copy())
#     dfEA = clip_roadnetwork(dfEA.copy())
#     dfEV = clip_roadnetwork(dfEV.copy())
#     dfMD = clip_roadnetwork(dfMD.copy())
#     # add TOD information
#     dfAM = add_TOD_information(dfAM.copy(),"AM")
#     dfPM = add_TOD_information(dfPM.copy(),"PM")
#     dfEA = add_TOD_information(dfEA.copy(),"EA")
#     dfEV = add_TOD_information(dfEV.copy(),"EV")
#     dfMD = add_TOD_information(dfMD.copy(),"MD")
#     # concat dataframes
#     df = pd.concat([dfAM,dfPM,dfEA,dfEV,dfMD])
#     df.reset_index(drop=True,inplace=True)
#     # get total capacity on the link for the day
#     df = getTotalCapacity(df.copy())
#     df = check_required_rdntwrk_colmns(df.copy())
#     # get Tot_Vol: by adding the columns which together form Tot_Vol
#     add_for_Tot_Vol = ['V1_1', 'V2_1', 'V3_1', 'V4_1', 'V5_1', 'V6_1', 'V7_1', 'V8_1', 'V9_1', 'V10_1', 'V11_1', 'V12_1',
#                        'V13_1','V14_1','V15_1','V16_1','V17_1','V18_1','V19_1',
#                        'BUSVOL_AM','BUSVOL_PM','BUSVOL_EA','BUSVOL_MD','BUSVOL_EV','OOS']
#     df["Tot_Vol"] = df[add_for_Tot_Vol].sum(axis=1)
#     # get Tot_TNC_Vol: ['V13_1','OOS'] # V13_1 is TNC_Volumes plying on the road segment
#     add_TNC_col = ['V13_1','OOS'] # V13_1 is TNC_Volumes plying on the road segment
#     df["Tot_TNC_Vol"] = df[add_TNC_col].sum(axis=1)
#     return df
#
# def agg_roadnetwrk(_dfmerged):
#     # aggregate the dataframe using A_B
#     wt_avg = lambda x: np.ma.average(x, weights = _dfmerged.loc[x.index, "Tot_Vol"],axis=0)
#
#     # aggregate function
#     def agg_func(_dfagg):
#         lst_col = ["SPEED","TIME","TIME_1","CSPD_1"]
#         # average the columns
#         avg_col = [ 'DISTANCE',
#                     "FT","AT",
#                     'TIMESEED',
#                     'LANE_AM', 'LANE_OP', 'LANE_PM', 'BUSLANE_AM', 'BUSLANE_OP', 'BUSLANE_PM',
#                     'TOLLAM_DA', 'TOLLAM_SR2', 'TOLLAM_SR3', 'TOLLPM_DA', 'TOLLPM_SR2', 'TOLLPM_SR3',
#                     'TOLLEA_DA', 'TOLLEA_SR2', 'TOLLEA_SR3', 'TOLLMD_DA', 'TOLLMD_SR2', 'TOLLMD_SR3', 'TOLLEV_DA',
#                     'TOLLEV_SR2', 'TOLLEV_SR3',"USE"]
#         d = {}
#         for col in _dfagg.select_dtypes(np.number).columns:
#             if col in lst_col:
#                 d[col] = wt_avg
#             elif col in avg_col:
#                 d[col]="mean"
#             else:
#                 d[col] = "sum"
#         for col in _dfagg.select_dtypes(object).columns:
#             d[col] = "first"
#         d["geometry"] = "first"
#         return d
#
#     _dfmerged["A_B"] = _dfmerged["A"].astype(str)  + "_" + _dfmerged["B"].astype(str)
#     _dfmerged["A"] = _dfmerged["A"].astype(str)
#     _dfmerged["B"] = _dfmerged["B"].astype(str)
#
#     dfmerged_agg = _dfmerged.groupby(['A_B'],as_index=False).aggregate(agg_func(_dfmerged.copy())).copy()
#
#     return dfmerged_agg
#
# # Network for YR 2010
# dfsfrd2010 = merge_TOD_dfs(gpd.read_file(BASE_DIR.parent.joinpath("2010","2010_AM.shp")),
#                               gpd.read_file(BASE_DIR.parent.joinpath("2010","2010_AM.shp")),
#                               gpd.read_file(BASE_DIR.parent.joinpath("2010","2010_EA.shp")),
#                               gpd.read_file(BASE_DIR.parent.joinpath("2010","2010_EV.shp")),
#                               gpd.read_file(BASE_DIR.parent.joinpath("2010","2010_MD.shp"))
#                               )
#
# dfsfrd2010 = agg_roadnetwrk(dfsfrd2010)
# # above merge converts the geo-dataframe to pandas dataframe. So re-convert it into geodataframe
# dfsfrdntwrk2010_agg = gpd.GeoDataFrame(dfsfrd2010, geometry='geometry',crs="EPSG:4326")
# dfsfrdntwrk2010_agg = dfsfrdntwrk2010_agg.to_crs("EPSG:4326")
# dfsfrdntwrk2010_agg = dfsfrdntwrk2010_agg.to_crs("EPSG:3857")
# dfsfrdntwrk2010_agg.to_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2010_PCS.geojson"), driver='GeoJSON')
#
# # Network for YR 2016
# dfsfrd2016 = merge_TOD_dfs(gpd.read_file(BASE_DIR.parent.joinpath("2016","2016_AM.shp")),
#                            gpd.read_file(BASE_DIR.parent.joinpath("2016","2016_AM.shp")),
#                            gpd.read_file(BASE_DIR.parent.joinpath("2016","2016_EA.shp")),
#                            gpd.read_file(BASE_DIR.parent.joinpath("2016","2016_EV.shp")),
#                            gpd.read_file(BASE_DIR.parent.joinpath("2016","2016_MD.shp"))
#                            )
#
# dfsfrd2016 = agg_roadnetwrk(dfsfrd2016)
# # above merge converts the geo-dataframe to pandas dataframe. So re-convert it into geodataframe
# dfsfrdntwrk2016_agg = gpd.GeoDataFrame(dfsfrd2016, geometry='geometry',crs="EPSG:4326")
# dfsfrdntwrk2016_agg = dfsfrdntwrk2016_agg.to_crs("EPSG:4326")
# dfsfrdntwrk2016_agg = dfsfrdntwrk2016_agg.to_crs("EPSG:3857")
# dfsfrdntwrk2016_agg.to_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2016_PCS.geojson"), driver='GeoJSON')

In [84]:
# # read the files
dfsfrdntwrk2010_agg = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2010_PCS.geojson"))
dfsfrdntwrk2016_agg = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2016_PCS.geojson"))

# intersect the road network with CensusTract file
def intersect_CT_rdntwrk(_gpdCT, _gpdrdntwrkagg):
    _gpdCT.to_crs(3857, inplace=True) # set its projection to EPSG:3857
    # _gpdrdntwrkagg.to_crs(3857,inplace=True)
    res_intersection = _gpdrdntwrkagg.overlay(_gpdCT, how='intersection')
    return res_intersection

def drop_intersect_CT_rdntwrk_columns(_df):
    df = _df.copy()
    cols = ['TOLL', 'USE', 'AT','LANE_AM', 'LANE_OP', 'LANE_PM',
            'BUSLANE_AM', 'BUSLANE_OP', 'BUSLANE_PM',
            'TOLLAM_DA', 'TOLLAM_SR2', 'TOLLAM_SR3', 'TOLLPM_DA', 'TOLLPM_SR2', 'TOLLPM_SR3', 'TOLLEA_DA', 'TOLLEA_SR2', 'TOLLEA_SR3', 'TOLLMD_DA', 'TOLLMD_SR2', 'TOLLMD_SR3', 'TOLLEV_DA', 'TOLLEV_SR2', 'TOLLEV_SR3',
            'VALUETOLL_', 'PASSTHRU',
            'BUSTPS_AM', 'BUSTPS_OP', 'BUSTPS_PM', 'TSVA', 'BIKE_CLASS', 'PER_RISE', 'ONEWAY', 'DTA_EDIT_F', 'TOLLTIME', 'PHASE',
            'AMBUSSAVE', 'MDBUSSAVE', 'PMBUSSAVE', 'EVBUSSAVE', 'EABUSSAVE',
            'SPDC', 'CAPC', 'A', 'B', 'STREETNAME', 'TYPE', 'MTYPE', 'TSIN', 'PROJ', 'ACTION', 'AB', 'peak', 'statefp10', 'mtfcc10', 'name10', 'intptlat10', 'awater10', 'namelsad10', 'funcstat10', 'aland10', 'geoid10', 'intptlon10', 'countyfp10',
            "V1T_1",'V2T_1', 'V3T_1',  'V4T_1', 'V5T_1','V6T_1', 'V7T_1','V8T_1', 'V9T_1',  'V10T_1', 'V11T_1', 'V12T_1', 'V13T_1', 'V14T_1','V15T_1', 'V16T_1', 'V17T_1','V18T_1', 'V19T_1', ]
    df = df.drop([x for x in cols if x in df.columns], axis=1)
    return df

gdfsfct = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SF_CensusTract_PCS.geojson"))

gdfsfrd2010ct_int = intersect_CT_rdntwrk(gdfsfct,dfsfrdntwrk2010_agg)
gdfsfrd2010ct_int = drop_intersect_CT_rdntwrk_columns(gdfsfrd2010ct_int)
gdfsfrd2010ct_int.to_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2010_CT_PCS.geojson"), driver='GeoJSON')

gdfsfrd2016ct_int = intersect_CT_rdntwrk(gdfsfct,dfsfrdntwrk2016_agg)
gdfsfrd2016ct_int =drop_intersect_CT_rdntwrk_columns(gdfsfrd2016ct_int)
gdfsfrd2016ct_int.to_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2016_CT_PCS.geojson"), driver='GeoJSON')

C:\Users\Goyal\.virtualenvs\Overpass_Turbo-ZwXzihL_\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\Goyal\.virtualenvs\Overpass_Turbo-ZwXzihL_\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [85]:
# Check in QGIS if the re-projection is successful .i.e.
# 1. Both SFChamp_2010_agg.geojson and SFChamp_2016_agg.geojson into EPSG:3857, are named with _PCS suffix
# 2. The SF_CensusTract to EPSG:3857, is also name with _PCS suffix

# After the above process do the following in QGIS
# 3. Road Crash for each year i.e. 2010 and 2016, convert it to EPSG:3857, name it SFCrash_2010_PCS.geojson & SFCrash_2016_PCS.geojson
# 4. Perform spatial intersection:
# Intersect with SF_CensusTract and RoadNetwork and name the output as SFChamp_201x_agg_CT_PCS.geojson
# Intersect road crashes with SF_Census Tract and name the output as SFCrash_201x_CT_PCS.geojson
# for both, keep "tractce" column from SF_Census Tract in the output file
# This ends QGIS manipulation

In [86]:
"""
We are grouping the crashes at two-levels
1. censustract/taz level
2. depending upon facility type

So,
the road crashes should be aggregated on
1. find the nearest link to which the crash could be attached (fld: A_B and D2NL<10)
2. create a unique identifier using  censustract_ID & FT of roadnetwork: tractce10_FT
3. aggregate all the road crashes attached to tractce10_FT
4. aggregate all road network attached to tractce10_FT
"""

'\nWe are grouping the crashes at two-levels\n1. censustract/taz level\n2. depending upon facility type\n\nSo,\nthe road crashes should be aggregated on\n1. find the nearest link to which the crash could be attached (fld: A_B and D2NL<10)\n2. create a unique identifier using  censustract_ID & FT of roadnetwork: tractce10_FT\n3. aggregate all the road crashes attached to tractce10_FT\n4. aggregate all road network attached to tractce10_FT\n'

In [87]:
# modify the dataframe to create a new column "CATEGORY" using "FacilityType"
def label_df_by_road_category(_df,fld):
    _df["category"]=0
    _df.loc[_df[fld].isin([1, 2, 3, 5,13]),'category']=1
    _df.loc[_df[fld].isin([4,7,12,15]),'category']=2
    _df.loc[_df[fld].isin([9,11 ]),'category']=3
    return _df

def add_unique_ID_using_CT_CAT(_df,CT_ID_fld,CAT_fld):
    d = {}
    if isinstance(_df,gpd.GeoDataFrame):
        d[f"{CAT_fld}_{CT_ID_fld}"] = _df[CT_ID_fld].astype(str) + "_" + _df[CAT_fld].astype(int).astype(str)
    return pd.concat([_df, pd.DataFrame(d, index=_df.index)],axis=1)

# Fetch SF_Census Tract
SF_CT = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SF_CensusTract_PCS.geojson"), crs = "EPSG:3857")
SF_CT = SF_CT.to_crs(3857)

# read the merged road network files and containing CensusTract IDs
gdfsfrd2010ct_int= gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2010_CT_PCS.geojson"))
gdfsfrd2016ct_int= gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"SFChamp_2016_CT_PCS.geojson"))

# categorize the road network by FacilityType
gdfsfrd2010ct = label_df_by_road_category(gdfsfrd2010ct_int.copy(),"FT")
gdfsfrd2016ct = label_df_by_road_category(gdfsfrd2016ct_int.copy(),"FT")

gdfsfrd2010ct_cat = add_unique_ID_using_CT_CAT(gdfsfrd2010ct.copy(),"tractce10","category")
gdfsfrd2010ct_cat.replace([np.inf, -np.inf], np.nan, inplace=True)
gdfsfrd2010ct_cat.fillna(0, inplace=True)

gdfsfrd2016ct_cat = add_unique_ID_using_CT_CAT(gdfsfrd2016ct.copy(),"tractce10","category")
gdfsfrd2016ct_cat.replace([np.inf, -np.inf], np.nan, inplace=True)
gdfsfrd2016ct_cat.fillna(0, inplace=True)

In [88]:
# read the (Nearest Neighbour) road crash files and containing CensusTract IDs
gdfsfnncrash2010ct = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"NN_SFCrash_2010_CT_PCS.geojson"))
gdfsfnncrash2016ct = gpd.read_file(BASE_DIR.parent.joinpath(folder_path,"NN_SFCrash_2016_CT_PCS.geojson"))

# categorize the road crash by FacilityType
gdfsfrdcrash2010ct_cat = label_df_by_road_category(gdfsfnncrash2010ct.copy(),"FT")
gdfsfrdcrash2016ct_cat = label_df_by_road_category(gdfsfnncrash2016ct.copy(),"FT")

# drop tractce10 and rename join_tractce10
gdfsfrdcrash2010ct_cat = gdfsfrdcrash2010ct_cat.loc[:,gdfsfrdcrash2010ct_cat.columns!="tractce10"].copy()
gdfsfrdcrash2010ct_cat.rename(columns={"join_tractce10":"tractce10"},inplace=True)
gdfsfrdcrash2016ct_cat = gdfsfrdcrash2016ct_cat.loc[:,gdfsfrdcrash2016ct_cat.columns!="tractce10"].copy()
gdfsfrdcrash2016ct_cat.rename(columns={"join_tractce10":"tractce10"},inplace=True)

def drop_crash_columns(_df):
    df = _df.copy()
    cols = ['ACCIDENT_YEAR', 'PROC_DATE', 'JURIS', 'COLLISION_DATE', 'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT', 'DAY_OF_WEEK', 'CHP_SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND', 'BEAT_TYPE', 'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS', 'BEAT_NUMBER', 'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION', 'INTERSECTION', 'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND', 'CALTRANS_COUNTY', 'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX', 'POSTMILE_PREFIX', 'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION', 'SIDE_OF_HWY', 'TOW_AWAY', 'COLLISION_SEVERITY',  'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_OF_VIOL', 'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION', 'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION', 'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING', 'CONTROL_DEVICE', 'CHP_ROAD_TYPE', 'PEDESTRIAN_ACCIDENT', 'BICYCLE_ACCIDENT', 'MOTORCYCLE_ACCIDENT', 'TRUCK_ACCIDENT', 'NOT_PRIVATE_PROPERTY', 'ALCOHOL_INVOLVED', 'STWD_VEHTYPE_AT_FAULT', 'CHP_VEHTYPE_AT_FAULT', 'PRIMARY_RAMP', 'SECONDARY_RAMP', 'LATITUDE', 'LONGITUDE', 'COUNTY', 'CITY', 'POINT_X', 'POINT_Y', 'PRIMARY_RD_3', 'SECONDARY_RD_3', 'tractce10', 'FT',"CASE_ID"]
    df = df.drop([x for x in cols if x in df.columns], axis=1)
    return df

gdfsfrdcrash2010ct_cat = drop_crash_columns(add_unique_ID_using_CT_CAT(gdfsfrdcrash2010ct_cat,"tractce10","category"))
gdfsfrdcrash2016ct_cat = drop_crash_columns(add_unique_ID_using_CT_CAT(gdfsfrdcrash2016ct_cat,"tractce10","category"))

def min_D2NL(_df,dist):
    df = _df.loc[_df["D2NL"]<dist,:]
    return df

gdfsfrdcrash2010ct_cat = min_D2NL(gdfsfrdcrash2010ct_cat.copy(),10)
gdfsfrdcrash2016ct_cat = min_D2NL(gdfsfrdcrash2016ct_cat.copy(),10)

In [89]:
# aggregate road network by unqiueID: category_tractce10
def reqd_colmns(_df):# road network
    df = _df.copy()
    d = {}
    reqd_colmns = ['V1_1', 'V2_1', 'V3_1', 'V4_1', 'V5_1', 'V6_1', 'V7_1', 'V8_1', 'V9_1', 'V10_1', 'V11_1', 'V12_1', 'V13_1', 'V14_1', 'V15_1', 'V16_1', 'V17_1', 'V18_1', 'V19_1',
                   'OOS', 'PUDO',
                   'BUSVOL_AM', 'BUSVOL_AM', 'BUSVOL_MD', 'BUSVOL_PM', 'BUSVOL_EV', 'BUSVOL_EA',
                   'Tot_CAP', 'CAP', 'SPEED','CSPD_1',
                   'Tot_Vol', 'Tot_TNC_Vol', 'Tot_Non_TNC_Vol', 'TNC_Tot_Vol'
                   ]
    for col in reqd_colmns:
        if col not in df.columns:
            d[col]=0
    return pd.concat([df, pd.DataFrame(d, index=df.index)],axis=1)

def get_req_fields(_df):
    fields = ["Tot_Vol","Tot_TNC_Vol", "Tot_Non_TNC_Vol", "Tot_VMT","Tot_TNC_VMT","Tot_Non_TNC_VMT", "Congested_Speed",'CSPD_1',"SPEED","PUDO","OOS"]
    d = {}
    for fld in fields:
        if fld == "Tot_Vol":# TNC Tot Vol
            cols = ['V1_1', 'V2_1', 'V3_1', 'V4_1', 'V5_1', 'V6_1', 'V7_1', 'V8_1', 'V9_1', 'V10_1',
                    'V11_1', 'V12_1', 'V13_1', 'V14_1', 'V15_1', 'V16_1', 'V17_1', 'V18_1', 'V19_1',
                    "OOS",
                    'BUSVOL_AM', 'BUSVOL_AM', 'BUSVOL_MD', 'BUSVOL_PM', 'BUSVOL_EV', 'BUSVOL_EA',]
            d[fld] = _df[cols].sum(axis=1)
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Tot_TNC_Vol":# TNC Tot Vol
            cols = ['V13_1',"OOS"]
            d[fld] = _df[cols].sum(axis=1)
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Tot_Non_TNC_Vol":
            cols = ['V13_1',"OOS"]
            d[fld] = d["Tot_Vol"] - _df[cols].sum(axis=1)
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Tot_VMT":
            d[fld] = d["Tot_Vol"]*_df["Length_meters"]*0.000621371
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Tot_TNC_VMT":
            cols = ['V13_1',"OOS"]
            d[fld] = (_df[cols].sum(axis=1))*_df["Length_meters"]*0.000621371
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Tot_Non_TNC_VMT":
            cols = ['V13_1',"OOS"]
            d[fld] = (d["Tot_Vol"] - _df[cols].sum(axis=1))*_df["Length_meters"]*0.000621371
            d[f"{fld}_mil"] = d[fld].divide(1000000)
            d[f"{fld}_yr"] = d[fld]*365
            d[f"{fld}_mil_yr"] =  d[f"{fld}_yr"].divide(1000000)
            d[f"log_{fld}"] = np.log(d[fld]+1)
            d[f"log_{fld}_mil"] = np.log(d[f"{fld}_mil"]+1)
            d[f"log_{fld}_mil_yr"] = np.log(d[f"{fld}_mil_yr"]+1)
        elif fld == "Congested_Speed":
            d["Congested_Speed"] = (((_df["Length_meters"]*0.000621371).divide(_df["TIME_1"]))*60)
            d["Congested_Speed_yr"] = d["Congested_Speed"]
        elif fld == "CSPD_1":
            d["CSPD_1_yr"] = _df["CSPD_1"]
        elif fld == "SPEED":
            d["SPEED_yr"] = _df["SPEED"]
        elif fld == "PUDO":
            d["PUDO_yr"] = (_df["PUDO"]*365)
            d["PUDO_thousands"] = _df["PUDO"].divide(1000)
            d["PUDO_thousands_yr"] = d["PUDO_thousands"]*365
            d["PUDO_mil"] = _df["PUDO"].divide(1000000)
            d["PUDO_mil_yr"] = d["PUDO_yr"].divide(1000000)

            d["log_PUDO"] = np.log(_df["PUDO"]+1)
            d["log_PUDO_yr"] = np.log(d["PUDO_yr"]+1)
            d["log_PUDO_thousands"] = np.log(d["PUDO_thousands"]+1)
            d["log_PUDO_thousands_yr"] = np.log(d["PUDO_thousands_yr"]+1)
            d["log_PUDO_mil"] = np.log(d["PUDO_mil"]+1)
            d["log_PUDO_mil_yr"] = np.log(d["PUDO_mil_yr"]+1)
        elif fld == "OOS":
            d["OOS_yr"] = (_df["OOS"]*365)
            d["OOS_thousands"] = _df["OOS"].divide(1000)
            d["OOS_thousands_yr"] = d["OOS_thousands"]*365
            d["OOS_mil"] = _df["OOS"].divide(1000000)
            d["OOS_mil_yr"] = d["OOS_yr"].divide(1000000)

            d["log_OOS"] = np.log(_df["OOS"]+1)
            d["log_OOS_yr"] = np.log(d["OOS_yr"]+1)
            d["log_OOS_thousands"] = np.log(d["OOS_thousands"]+1)
            d["log_OOS_thousands_yr"] = np.log(d["OOS_thousands_yr"]+1)
            d["log_OOS_mil"] = np.log(d["OOS_mil"]+1)
            d["log_OOS_mil_yr"] = np.log(d["OOS_mil_yr"]+1)
    return pd.concat([_df, pd.DataFrame(d, index=_df.index)],axis=1)

def agg_network_by_uniqueIDs(_df,uniqueID):
    _df = reqd_colmns(_df.copy())
    _df.replace([np.inf, -np.inf], np.nan, inplace=True)
    _df.fillna(0, inplace=True)
    _df["FT"] = _df["FT"].astype(str)
    _df[uniqueID] = _df[uniqueID].astype(str)
    # aggregate the dataframe using A_B
    wt_avg = lambda x: np.ma.average(x, weights = _df.loc[x.index, "Tot_Vol"])
    # Aggregating rows based on one column with “, ”.join
    concat_agg = lambda ar: ', '.join([item for item in ar if item])
    def agg_func_rdntwrk(df):
        d = {}
        for col in df.select_dtypes(np.number).columns:
            if col in wt_col:
                d[col] = wt_avg
            else:
                d[col] = "sum"
        for col in df.select_dtypes(object).columns:
            if col in str_col:
                d[col] = "first"
            elif col=="FT":
                d[col] = concat_agg
        d["geometry"] = "first"
        return d

    wt_col = ["SPEED","TIME","CSPD_1", 'VDT_1', 'VHT_1','VC_1', ]
    str_col = ['category_tractce10',"FT"]
    drop_col = [ "A_B",'tractce10','category',]

    _df.drop(drop_col,axis=1,inplace=True)
    df = _df.groupby([uniqueID],as_index=False).aggregate(agg_func_rdntwrk(_df.copy())).copy()

    return get_req_fields(df)

def add_length_columns(_df,_yr):
    #remember to rename DISTANCE variable (as this is no longer the actual distance (in miles), given that feature is split-up)
    d = {}
    if isinstance(_df,gpd.GeoDataFrame):
        d["Length_meters"] = _df.geometry.length
        d["Length_miles"] = d["Length_meters"]* 0.000621371
        d["Year"] = _yr
    return pd.concat([_df, pd.DataFrame(d, index=_df.index)],axis=1)

gdfsfrd2010ct_cat_agg = agg_network_by_uniqueIDs(add_length_columns(gdfsfrd2010ct_cat.copy(),2010),"category_tractce10")
gdfsfrd2016ct_cat_agg = agg_network_by_uniqueIDs(add_length_columns(gdfsfrd2016ct_cat.copy(),2016),"category_tractce10")

C:\Users\Goyal\.virtualenvs\Overpass_Turbo-ZwXzihL_\lib\site-packages\numpy\ma\extras.py:623: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
C:\Users\Goyal\.virtualenvs\Overpass_Turbo-ZwXzihL_\lib\site-packages\numpy\ma\extras.py:623: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl


In [90]:
# aggregate crashes by uniqueID: category_tractce10
def agg_crash_by_uniqueIDs(_df,uniqueID):
    _df[uniqueID] = _df[uniqueID].astype(str)
    str_col = [uniqueID]
    # Aggregating rows based on one column with “, ”.join
    concat_agg = lambda ar: ', '.join([item for item in ar if item])
    def agg_func(df):
        d = {}
        for col in df.select_dtypes(np.number).columns:
            d[col] = "sum"
        for col in df.select_dtypes(object).columns:
            if col in str_col:
                d[col] = "first"
            else:
                d[col] = concat_agg
        return d
    df = _df.groupby([uniqueID],as_index=False).aggregate(agg_func(_df.copy())).copy()
    return df

gdfsfrdcrash2010ct_cat_agg = agg_crash_by_uniqueIDs(gdfsfrdcrash2010ct_cat,"category_tractce10")
gdfsfrdcrash2010ct_cat_agg["Accident_Year"] = 2010
gdfsfrdcrash2016ct_cat_agg = agg_crash_by_uniqueIDs(gdfsfrdcrash2016ct_cat,"category_tractce10")
gdfsfrdcrash2016ct_cat_agg["Accident_Year"] = 2016

In [91]:
def dataframe_merge(_dfroadnetwork, _dfroadcrash,left_fld,right_fld):
    dfmerge = pd.merge(_dfroadnetwork,_dfroadcrash,left_on=left_fld,right_on=right_fld,how="left")
    return dfmerge

# def update_fields(_df,_yr):
#     df = _df.copy()
#     if isinstance(df,pd.DataFrame):
#         df["Accident_Year"] = _yr
#         df["Crash_Year"] = _yr
#         d = {**dict.fromkeys(df.select_dtypes(np.number).columns, 0),
#              **dict.fromkeys(df.select_dtypes(exclude=np.number).columns,np.nan)}
#         df.fillna(d, inplace=True)
#     return pd.concat([df, pd.DataFrame(d, index=_df.index)],axis=1)

dfsf_rd_ntwrk_crash_2010_cat_agg = dataframe_merge(gdfsfrd2010ct_cat_agg,gdfsfrdcrash2010ct_cat_agg,"category_tractce10","category_tractce10")
d = {**dict.fromkeys(dfsf_rd_ntwrk_crash_2010_cat_agg.select_dtypes(np.number).columns, 0),
     **dict.fromkeys(dfsf_rd_ntwrk_crash_2010_cat_agg.select_dtypes(exclude=np.number).columns,np.nan)}
dfsf_rd_ntwrk_crash_2010_cat_agg =dfsf_rd_ntwrk_crash_2010_cat_agg.fillna(d)
dfsf_rd_ntwrk_crash_2010_cat_agg["Accident_Year"]=2010
dfsf_rd_ntwrk_crash_2016_cat_agg = dataframe_merge(gdfsfrd2016ct_cat_agg,gdfsfrdcrash2016ct_cat_agg,"category_tractce10","category_tractce10")
d = {**dict.fromkeys(dfsf_rd_ntwrk_crash_2016_cat_agg.select_dtypes(np.number).columns, 0),
     **dict.fromkeys(dfsf_rd_ntwrk_crash_2016_cat_agg.select_dtypes(exclude=np.number).columns,np.nan)}
dfsf_rd_ntwrk_crash_2016_cat_agg =dfsf_rd_ntwrk_crash_2016_cat_agg.fillna(d)
dfsf_rd_ntwrk_crash_2016_cat_agg["Accident_Year"]=2016


df2010 = dfsf_rd_ntwrk_crash_2010_cat_agg.loc[:,~dfsf_rd_ntwrk_crash_2010_cat_agg.columns.isin(["geometry"])].copy()
df2016 = dfsf_rd_ntwrk_crash_2016_cat_agg.loc[:,~dfsf_rd_ntwrk_crash_2016_cat_agg.columns.isin(["geometry"])].copy()
df2010.reset_index(drop=True, inplace=True)
df2016.reset_index(drop=True, inplace=True)
columns_to_retain = set(df2010.columns.to_list()).intersection(set(df2016.columns.to_list()))
dfmerged =pd.concat([df2010[columns_to_retain], df2016[columns_to_retain]], ignore_index=True,verify_integrity=True,copy=True,axis=0)
# few cosmetic changes to read the dataframe better
def add_custom_fields(_df):
    df = _df.copy()
    df = df.sort_index(axis=1).sort_values(by=["category_tractce10", "Accident_Year"])
    df.reset_index(drop=True, inplace=True)
    df.insert(0, "category_tractce10", df.pop("category_tractce10"))
    df[["tractce10", "category"]] = df["category_tractce10"].str.split("_", 1, expand=True)
    df.insert(1, "tractce10", df.pop("tractce10"))
    df.insert(3, "category", df.pop("category"))
    df = df.assign(cat_1=0, cat_2=0, cat_3=0, vision_zero=0)
    df.loc[df["category"] == 1, "cat_1"]=1
    df.loc[df["category"] == 2, "cat_2"]=1
    df.loc[df["category"] == 3, "cat_3"]=1
    df.loc[df["Accident_Year"] == 2016, "Vision_Zero"]=1
    d= {}
    d["SPD_ratio"] = df["CSPD_1"].divide(df["SPEED"])
    d["PUDO_pct_Tot_TNC_VMT"] = df["PUDO"].divide(df["Tot_TNC_VMT"])
    d["PUDO_pct_Tot_TNC_VMT_yr"] = df["PUDO_yr"].divide(df["Tot_TNC_VMT_yr"])
    d["COUNT_Fatal_and_Injury"] = df["COUNT_Fatal"] + df["COUNT_Visible_Injury"] + df["COUNT_Severe_Injury"]+ df["COUNT_Other_Injury"]
    d["TotCrash_permile"] = df["Total_Crash"].divide(df["Length_miles"])
    d["Tot_FatalInj_permile"] = d["COUNT_Fatal_and_Injury"].divide(df["Length_miles"])
    d["Tot_TNC_VMT_permile"] = df["Tot_TNC_VMT"].divide(df["Length_miles"])
    d["log_Tot_TNC_VMT_permile"] = np.log(d["Tot_TNC_VMT_permile"]+1)
    d["Tot_Non_TNC_VMT_permile"] = df["Tot_Non_TNC_VMT"].divide(df["Length_miles"])
    d["log_Tot_Non_TNC_VMT_permile"] = np.log(d["Tot_Non_TNC_VMT_permile"]+1)
    d["PUDO_permile"] = df["PUDO"].divide(df["Length_miles"])
    d["log_PUDO_permile"] = np.log(d["PUDO_permile"]+1)
    return pd.concat([df, pd.DataFrame(d, index=df.index)],axis=1)

dfmerged_mod = add_custom_fields(dfmerged)
dfmerged_mod.replace([np.inf, -np.inf], np.nan, inplace=True)
dfmerged_mod.fillna(0,inplace=True)

# dfmerged = dfmerged.sort_index(axis=1).sort_values(by=["category_tractce10","Accident_Year"])
# dfmerged.reset_index(drop=True,inplace=True)
# dfmerged.insert(0,"category_tractce10",dfmerged.pop("category_tractce10"))
# dfmerged[["tractce10", "category"]] = dfmerged["category_tractce10"].str.split("_",1,expand=True)
# dfmerged.insert(1,"tractce10",dfmerged.pop("tractce10"))
# dfmerged.insert(3,"category",dfmerged.pop("category"))
# dfmerged = dfmerged.assign(cat_1=0,cat_2=0,cat_3=0,vision_zero=0)
# dfmerged.loc[dfmerged["category"]==1,"cat_1"]=1
# dfmerged.loc[dfmerged["category"]==2,"cat_1"]=2
# dfmerged.loc[dfmerged["category"]==3,"cat_1"]=3
# dfmerged.loc[dfmerged["Accident_Year"]==2016,"vision_zero"]=1
# dfmerged["SPD_ratio"] = dfmerged["CSPD_1"].divide(dfmerged["SPEED"])
# dfmerged["PUDO_pct_Tot_TNC_VMT"] = dfmerged["PUDO"].divide(dfmerged["Tot_TNC_VMT"])
dfmerged_mod.to_csv(BASE_DIR.parent.joinpath(folder_path,"SF_merged_CAT_CT.csv"))

C:\Users\Goyal\AppData\Local\Temp\ipykernel_18568\1867483328.py:32: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dfmerged =pd.concat([df2010[columns_to_retain], df2016[columns_to_retain]], ignore_index=True,verify_integrity=True,copy=True,axis=0)
C:\Users\Goyal\AppData\Local\Temp\ipykernel_18568\1867483328.py:32: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dfmerged =pd.concat([df2010[columns_to_retain], df2016[columns_to_retain]], ignore_index=True,verify_integrity=True,copy=True,axis=0)


In [59]:
dfmerged.insert(0,"category_tractce10",dfmerged.pop("category_tractce10"))

In [61]:
dfmerged = dfmerged.sort_index(axis=1).sort_values(by=["category_tractce10","Accident_Year"])
dfmerged.reset_index(drop=True,inplace=True)
dfmerged

,category_tractce10,Accident_Year,BUSVOL_AM,BUSVOL_EA,BUSVOL_EV,BUSVOL_MD,BUSVOL_PM,CAP,COUNT_BICYCLIST_INJURED,COUNT_BICYCLIST_KILLED,...,log_Tot_TNC_VMT_mil_yr,log_Tot_TNC_Vol,log_Tot_TNC_Vol_mil,log_Tot_TNC_Vol_mil_yr,log_Tot_VMT,log_Tot_VMT_mil,log_Tot_VMT_mil_yr,log_Tot_Vol,log_Tot_Vol_mil,log_Tot_Vol_mil_yr
0,010100_2,2010,11514.75,2993.6,11424.0,17123.0,13249.2,373500,9.0,0.0,...,0.000000,0.000000,0.000000,0.000000,15.143741,1.563259,7.228853,12.913454,0.340559,5.004568
1,010100_2,2016,11514.75,2993.6,11424.0,17123.0,13249.2,385500,13.0,0.0,...,5.564499,11.197776,0.070428,3.319009,15.410032,1.779345,7.494975,13.131523,0.408527,5.221323
2,010100_3,2010,1680.00,723.6,4250.0,4463.2,2220.0,159000,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,13.270761,0.457416,5.359858,11.496418,0.093820,3.608267
3,010100_3,2016,1680.00,723.6,4250.0,4463.2,2220.0,159000,0.0,0.0,...,3.551347,9.663567,0.015610,1.908427,13.403585,0.508246,5.492098,11.629241,0.106461,3.737716
4,010200_1,2010,0.00,0.0,0.0,0.0,0.0,990,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,980900_1,2016,697.50,0.0,0.0,0.0,0.0,113960,0.0,0.0,...,3.980938,10.431795,0.033358,2.593826,14.164770,0.882948,6.251087,12.718832,0.288165,4.811385
902,980900_2,2010,8822.50,5738.6,14492.5,17972.5,8790.0,651250,6.0,0.0,...,0.000000,0.000000,0.000000,0.000000,15.942779,2.239849,8.027492,13.398592,0.506259,5.487126
903,980900_2,2016,8822.50,5738.6,14492.5,17972.5,8790.0,651250,2.0,0.0,...,5.627785,10.995623,0.057902,3.124928,16.230872,2.500929,8.315504,13.686685,0.630807,5.774182
904,980900_3,2010,4215.00,3484.4,6332.5,8287.5,4105.0,538750,6.0,0.0,...,0.000000,0.000000,0.000000,0.000000,15.719732,2.043060,7.804526,12.242554,0.188494,4.340057


In [62]:
dfmerged[["tractce10", "category"]] = dfmerged["category_tractce10"].str.split("_",1,expand=True)

In [64]:
dfmerged.insert(1,"tractce10",dfmerged.pop("tractce10"))
dfmerged.insert(3,"category",dfmerged.pop("category"))

In [ ]:
dfmerged = dfmerged.assign(cat_1=0,cat_2=0,cat_3=0,vision_zero=0)